Alright. I'm subscribed. Get my 9$ a month qworth!


https://www.computerenhance.com/p/table-of-contents


It was really interesting seeing casey muratoi 
https://www.computerenhance.com/p/the-case-of-the-missing-increment

https://github.com/cmuratori/computer_enhance

In [3]:
def SingleScalar(Count, Input):
    Sum = 0
    for Index in range(0, Count):
        Sum += Input[Index]
    return Sum


freq = 3e9 # a guess at 3ghz
def cycles(t):
    return t * freq
import time

mintime = 10000
data = [1]*4096
for i in range(10):
    now = time.time()
    SingleScalar(4096,data)
    mintime = min(mintime, time.time()-now)
print(mintime)
print(cycles(mintime))
print("cycle per add:", cycles(mintime)/4096)



0.0001232624053955078
369787.21618652344
cycle per add: 90.28008207678795


In [1]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         48 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  16
  On-line CPU(s) list:   0-15
Vendor ID:               AuthenticAMD
  Model name:            AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics
    CPU family:          25
    Model:               116
    Thread(s) per core:  2
    Core(s) per socket:  8
    Socket(s):           1
    Stepping:            1
    CPU max MHz:         6076.0000
    CPU min MHz:         400.0000
    BogoMIPS:            6586.92
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall n
                         x mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_go
                         od amd_lbr_v2 nopl nonstop_tsc cpuid extd_apicid aperfm
                         perf rapl pni pclmulqdq monitor ssse3 fma cx16 

In [ ]:
!cat /proc/cpuinfo

In [15]:
%%file /tmp/SingleScalar.c

#include <stdio.h>
#include <time.h>

typedef unsigned int u32;
__attribute__((noinline)) 
u32 SingleScalar(u32 Count, u32 *Input)
{
	u32 Sum = 0;
	for(u32 Index = 0; Index < Count; ++Index)
	{
		Sum += Input[Index];
	}
	
	return Sum;
}


int main(){
    u32 data[4096];
	for (int i = 0; i < 4096; i++){
		data[i] = 1;
	}
	int mintime = 100000;
    

    for(int i = 0; i < 10; i++){
		clock_t start = clock();
		SingleScalar(4096, data);
		clock_t end = clock();
		int time = (int)(end - start);
		mintime = mintime < time ? mintime : time;
	}
    printf("%d\n", mintime);
    
	return 0;
}

Overwriting /tmp/SingleScalar.c


In [20]:
!objdump -d /tmp/SingleScalar | grep -A 20 "<SingleScalar>:"

0000000000001169 <SingleScalar>:
    1169:	f3 0f 1e fa          	endbr64 
    116d:	85 ff                	test   %edi,%edi
    116f:	74 1c                	je     118d <SingleScalar+0x24>
    1171:	48 89 f2             	mov    %rsi,%rdx
    1174:	8d 47 ff             	lea    -0x1(%rdi),%eax
    1177:	48 8d 4c 86 04       	lea    0x4(%rsi,%rax,4),%rcx
    117c:	b8 00 00 00 00       	mov    $0x0,%eax
    1181:	03 02                	add    (%rdx),%eax
    1183:	48 83 c2 04          	add    $0x4,%rdx
    1187:	48 39 ca             	cmp    %rcx,%rdx
    118a:	75 f5                	jne    1181 <SingleScalar+0x18>
    118c:	c3                   	ret    
    118d:	89 f8                	mov    %edi,%eax
    118f:	c3                   	ret    

0000000000001190 <main>:
    1190:	f3 0f 1e fa          	endbr64 
    1194:	41 54                	push   %r12
    1196:	55                   	push   %rbp
    1197:	53                   	push   %rbx


In [19]:
! gcc -O1 -o /tmp/SingleScalar /tmp/SingleScalar.c && /tmp/SingleScalar

0
